# Setup

## Importação de Bibliotecas

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
import seaborn as sns

np.random.seed(0)

## Importação dos Dados

In [ ]:
!pip install gdown

Download do arquivo pelo link disponibilizado no Drive

In [ ]:
!gdown --id '1TDHXj0Gb9HkztdeAjLmGjYTcxaJn0-it'

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1TDHXj0Gb9HkztdeAjLmGjYTcxaJn0-it 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# Substitua 'nome_do_arquivo.tsv' pelo nome do seu arquivo
data = pd.read_excel('/content/drive/MyDrive/Inteli/M6/Database/bd_limpo.xlsx', engine='openpyxl')
data.head(100)

# Análise Exploratória

Dimensão da Base

In [ ]:
# Número de Linhas e Colunas
print(f'Linhas: {data.shape[0]}; Colunas: {data.shape[1]}' )

Nomes e tipos das colunas:

In [ ]:
data.dtypes

Renomeia a coluna de avaliação.

In [ ]:
data.rename(columns = {0: 'Avaliação'}, inplace = True)

Converte a coluna de quantidade

In [ ]:
data["Qtde."] = pd.to_numeric(data["Qtde."], errors='coerce')

### Estatísticas Descritivas

In [ ]:
data[data["Avaliação"]>0].describe().round(2)

Considerando apenas as avaliações entre 1 e 5, vemos que as avalições tem média de 3.43 e mediana de 4.

### Distribuição das Avaliações

In [ ]:
sns.displot(data=data, x="Avaliação")

Complementamos as estatísticas descritivas das Avaliações com um gráfico de distribuição. Vemos como a avaliação mais comum (moda) é a nota 5, seguida de 4. Logo, as avaliações observadas (desconsiderando o 0) tendem a ser mais positivas.

Notamos também que o valor mais comum depois do 5 e 4 é o próprio 0.

### Padrões de Classificação

Vemos primeiro a distribuição da avaliação *média* por Escola.

In [ ]:
sns.displot(data=data[data["Avaliação"]>0].groupby('Escola')['Avaliação'].mean().to_frame(), x="Avaliação")

Vemos agora a distribuição do desvio padrão da Avaliação por Escola, como medida de dispersão das notas dadas em cada Escola.

In [ ]:
sns.displot(data=data[data["Avaliação"]>0].groupby('Escola')['Avaliação'].std().to_frame(), x="Avaliação")

Vemos certo grau de dispersão na maioria das escolas (desvio padrão acima entre 1 e 2), enquanto cerca de pouco menos de 400 escolas tem desvio padrão 0, indicando apenas um valor constante de avaliação.

E também analisamos o grau de dispersão das avaliações de uma Escola para um dado Fornecedor e Item:

In [ ]:
sns.displot(data=data[data["Avaliação"]>0].groupby(['Escola', 'Fornecedor', 'Item'])['Avaliação'].std().to_frame(), x="Avaliação")

Vemos que o grau de dispersão é menor, mas ainda existente.

### Popularidade de Fornecedores/Itens:

Vemos a distribuição do número de avaliações por Fornecedor e por Item:

In [ ]:
sns.displot(data=data[data["Avaliação"]>0]['Fornecedor'].value_counts().to_frame(), x="Fornecedor", bins=20)

In [ ]:
sns.displot(data=data[data["Avaliação"]>0]['Item'].value_counts().to_frame(), x="Item")

In [ ]:
data[data["Avaliação"]>0]['Fornecedor'].value_counts().to_frame().describe()

In [ ]:
data[data["Avaliação"]>0]['Item'].value_counts().to_frame().describe()

In [ ]:
data[data["Avaliação"]>0]['Fornecedor'].value_counts().to_frame()

In [ ]:
data[data["Avaliação"]>0]['Item'].value_counts().to_frame()

Notamos alguns Fornecedores outliers, com mais de 3000 avaliações (aprox. Mediana + 2 x Desvio Padrão), como "RIO OFFICE COMÉRCIO DE MOVEIS E EQUIPAMENTOS EIREL", que possui mais de 13,000.

Também observamos Items com excesso de avaliações, como "viagens", com mais de 22,000 avaliações.

### Atividade das Escolas:

In [ ]:
data[data["Avaliação"]>0]['Escola'].value_counts().to_frame().describe()

In [ ]:
data[data["Avaliação"]>0]['Escola'].value_counts().to_frame()

Observamos algumas escolas outliers, com centenas de avaliações feitas, como é o caso da escola "MARIA DE LOURDES ALMEIDA SINISGALLI PROFA", com mais de 2,000 avaliações.

In [ ]:
data[data["Avaliação"]>0][['Escola', 'Fornecedor']].value_counts().to_frame()

Vemos acima também o número de avaliações para cada par Escola x Fornecedor. A escola com mais avaliações no total, realizou múltiplas avaliações para vários fornecedores.

In [ ]:
data[data["Avaliação"]>0][['Escola', 'Fornecedor', "Item"]].value_counts().to_frame()

Para um mesmo Item, vemos que uma mesma escola pode utilizar Fornecedores diferentes.

# Tratamento dos Dados

### Tratamento de Nulos



Percentual de nulos por coluna:

In [ ]:
data.isnull().mean() * 100

Se considerarmos os valores de Avaliação = 0 como nulos, vemos que este campo possui percentual elevado de nulos (mais de 20%).

In [ ]:
data['Avaliação'].replace(0, np.nan).isnull().mean() * 100

Tratamos aqui as Avalições = 0 como nulos e criamos uma nova base para processamento (df_processed), mantendo a original sem alterações:

In [ ]:
df_processed = data.copy()
df_processed['Avaliação'] = df_processed['Avaliação'].replace(0, np.nan)

In [ ]:
df_processed.isnull().mean() * 100

Dos demais campos de maior interesse (Escola e Fornecedor), vemos que o campo de Fornecedor possui número de nulos relativamente alto (13.2%).

Dado um Item e uma Escola, poderíamos imputar o valor do Fornecedor mais frequente ou mais recente, mas observamos acima que, para um mesmo Item, uma mesma escola pode utilizar Fornecedores diferentes.

Imputar um valor de nome da Escola é ainda mais arriscado, principalmente dado que o código de identificação das Escolas também é nulo na mesma frequência.

Portanto, seguimos excluindo quaisquer linhas que tenham o valor de Fornecedor ou Escola nulos.

In [ ]:
df_processed = df_processed.dropna(subset=['Fornecedor'])
df_processed = df_processed.dropna(subset=['Escola'])

In [ ]:
df_processed.isnull().mean() * 100

No caso da Avaliação, podemos preencher, quando possível, para uma combinação de Fornecedor, Escola e Item, a mediana da avaliação. Na Análise Exploratória, notamos que existe um grau de dispersão nas avaliações de uma Escola para um Item de um Fornecedor, mas em um grau um pouco menor.

In [ ]:
df_processed['Avaliação'] = df_processed['Avaliação'].fillna(df_processed.groupby(['Escola', 'Fornecedor', 'Item'])['Avaliação'].transform('median'))

In [ ]:
df_processed.isnull().mean() * 100

Conseguimos assim reduzir um pouco o percentual de avaliações nulas.

Removemos os nulos restantes de Avaliação:

In [ ]:
df_processed = df_processed.dropna(subset=['Avaliação'])

In [ ]:
print(f'Após o tratamento dos nulos, ficamos com {df_processed.shape[0]} observações')

In [ ]:
df_processed.isnull().mean() * 100

### Tratamento de outliers

In [ ]:
df_processed.describe()

Considerando como outlier os valores com distância de pelo menos 2 desvios padrões da média:

In [ ]:
df_processed[df_processed['Qtde.']>=(38.9+2*70.5)]

Valores extremos de quantidade parecem ser coerentes com o tipo de item, então vamos mantê-los.

Valores de avaliação estão coerentes, nenhum outlier fora do intervalo 1-5.

# Desenvolvimento do Modelo

## Filtragem Colaborativa por Escola

### Criação de Matriz

In [ ]:
## Cria dataframe de avaliação (Escola x Fornecedor), preenchendo NA com 0
pivot_table_mean = df_processed.pivot_table(values='Avaliação', index='Escola', columns='Fornecedor', aggfunc='mean', fill_value=0)
pivot_table_mean.sample(10)

#### Densidade da Matriz

Avaliação de densidade da matriz, considerando 0 como avaliação ausente:

In [ ]:
df_nulos = pd.DataFrame(pivot_table_mean.replace(0, np.nan).isnull().mean() * 100, columns=['PctNulos'])
sns.displot(data=df_nulos, x="PctNulos")

In [ ]:
df_nulos.describe()

Vemos que cerca de 83.8% da matrix Escola-Fornecedor é nula, indicando grau relativamente elevando de esparsidade (baixa densidade), sugerindo tratamento com técnicas como fatorização.

In [ ]:
## Normalizar subtraindo média de cada escola
df_normalizado = pivot_table_mean.sub(pivot_table_mean.mean(axis=1), axis=0)
df_normalizado

### Treino do Modelo

#### KNN

In [ ]:
# Usar o algoritmo KNN com a métrica de similaridade do cosseno, usando valor inicial de k=5 vizinhos
knn = NearestNeighbors(metric='cosine', n_neighbors=5, n_jobs=-1)

# Ajustar o modelo com os dados normalizados
knn.fit(df_normalizado)

# Calcular as distâncias e os índices dos vizinhos mais próximos para todos os usuários
distances, indices = knn.kneighbors(df_normalizado)

### Recomendação

In [ ]:
df_normalizado.sample(10)

In [ ]:
# Prevê a Avaliação que uma determinado Escola daria a um determinado Fornecedor.
def predict_rating(school_name, supplier_name, data, indices):

    school_index = np.where(data.index==school_name)[0][0]

    # Seleciona os índices das Escolas mais próximas para a Escola alvo.
    neighbor_indices = indices[school_index, 1:]  # Ignora o próprio usuário

    # Obtém as classificações que essas Escolas próximas deram para o Fernecedor especificado
    neighbor_ratings = data.loc[:, supplier_name].iloc[neighbor_indices]

    # Calcular a média das classificações dos vizinhos
    predicted_rating = neighbor_ratings.mean()

    return predicted_rating

In [ ]:
# Exemplo: Prever a Avaliação da Escola "PARQUE OZIEL" para o Fornecedor "WEBLABOR SÃO PAULO MATERIAIS DIDÁTICOS LTDA - EPP"
predict_rating('PARQUE OZIEL',
               'WEBLABOR SÃO PAULO MATERIAIS DIDÁTICOS LTDA - EPP',
               pivot_table_mean,
               indices)

In [ ]:
# Exemplo: Prever a Avaliação da Escola "GUSTAVO PECCININI" para o Fornecedor "WEBLABOR SÃO PAULO MATERIAIS DIDÁTICOS LTDA - EPP"
predict_rating('GUSTAVO PECCININI',
               'AGUAMAR TRANSPORTES LTDA',
               pivot_table_mean,
               indices)

### Avaliação dos Resultados

Reorganizamos a matriz Escola-Fornecedor de avaliações médias para ter cada combinação em um linha.

In [ ]:
df_pivot_reshape = pivot_table_mean.stack().reset_index(name='Avaliação')
df_pivot_reshape

Testando função no novo formato dos dados:

In [ ]:
df_pivot_reshape[(df_pivot_reshape['Escola']=="PARQUE OZIEL")&(df_pivot_reshape['Fornecedor']=="WEBLABOR SÃO PAULO MATERIAIS DIDÁTICOS LTDA - EPP")].apply(lambda x: predict_rating(x['Escola'], x['Fornecedor'], data=pivot_table_mean, indices=indices), axis=1)

As métricas de avaliação utilizadas, como RMSE (Root Mean Sqaured Error) e MAE (Mean Absolute Error), são comumente empregadas em modelos de recomendação para avaliar quão bem o modelo está performando em relação às previsões feitas em comparação as avaliações reais dos usuários. Focando em cada uma delas, temos:

RMSE (Root Mean Squared Erros):
- O RMSE é uma métrica que mede a raiz quadrada da média dos quadrados dos erros entre as previsões do modelo e as avaliações reais dos usuários. Valores menores de RMSE indicam que o modelo tem previsões mais precisas, sendo 0 o valor ideal (sem erro).

MAE (Mean Absolute Error):
- O MAE é uma métrica que calcula a média dos valores absolutos dos erros entre as previsões e as avaliações reais. Além disso, ele fornece uma medida da magnitude média dos erros, sem considerar a direção (subestimação ou superestimação).

Selecionamos uma amostra aleatória menor para os cálculos rodarem mais rapidamente no Colab:

In [ ]:
df_pivot_reshape_sample = df_pivot_reshape.sample(10000)

Geramos as previsões para cada combinação Escola-Fornecedor:

In [ ]:
df_pivot_reshape_sample['prediction'] = df_pivot_reshape_sample.apply(lambda x: predict_rating(x['Escola'], x['Fornecedor'], data=pivot_table_mean, indices=indices), axis=1)

E calculamos as métricas do erro da previsão do modelo de recomendação:

In [ ]:
print(f"RMSE = {mean_squared_error(df_pivot_reshape_sample['Avaliação'], df_pivot_reshape_sample['prediction'], squared=False)}")

In [ ]:
print(f"MAE = {mean_absolute_error(df_pivot_reshape_sample['Avaliação'], df_pivot_reshape_sample['prediction'])}")

# Exportando o modelo

Exportamos o modelo em formato pkl

In [ ]:
import joblib
joblib.dump(knn, 'knn_model.pkl')

Exportamos também a matriz de avaliação original e a normalizada para gerar as previsões.

In [ ]:
joblib.dump(pivot_table_mean, 'df_escola_fornecedor_mean.pkl')
joblib.dump(df_normalizado, 'df_normalizado_escola_fornecedor_mean.pkl')

# Utilizando o modelo


Podemos importar o modelo no formato pkl

In [ ]:
knn_saved = joblib.load('knn_model.pkl')

In [ ]:
distances_saved, indices_saved = knn_saved.kneighbors(df_normalizado)

E usamos a mesma função para gerar as previsões:

In [ ]:
predict_rating('PARQUE OZIEL',
               'WEBLABOR SÃO PAULO MATERIAIS DIDÁTICOS LTDA - EPP',
               pivot_table_mean,
               indices_saved)